<a href="https://colab.research.google.com/github/Akiyoshi-Yagi/GANs/blob/master/Semi_Supervised_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#諸々import
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K

from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Dropout, Flatten, Input, Lambda, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
#モデルの入力次元の設定

img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)
z_dim = 100
num_classes = 10

In [ ]:
#訓練とテストの為のデータの整形

class Dataset:
  def __init__(self, num_labeled):
    self.num_labeled = num_labeled
    (self.x_train, self.y_train),(self.x_test, self.y_test) = mnist.load_data()

    def preprocess_imgs(x):
      x = (x.astype(np.float32) - 127.5) /127.5
      #x = (x / 127.5) -1.0
      x = np.expand_dims(x, axis = 3)
      return x
    
    def preprocess_labels(y):
      return y.reshape(-1, 1)


    self.x_train = preprocess_imgs(self.x_train)
    self.y_train = preprocess_labels(self.y_train)

    self.x_test = preprocess_imgs(self.x_test)
    self.y_test = preprocess_labels(self.y_test)
  
  def batch_labeled(self, batch_size):
    idx = np.random.randint(0, self.num_labeled, batch_size)
    imgs = self.x_train[idx]
    labels = self.y_train[idx]
    return imgs, labels
  
  def batch_unlabeled(self, batch_size):
    idx = np.random.randint(self.num_labeled, self.x_train.shape[0], batch_size)
    imgs = self.x_train[idx]

    return imgs
  
  def training_set(self):
    x_train = self.x_train[range(self.num_labeled)]
    y_train = self.y_train[range(self.num_labeled)]
    return x_train, y_train
  
  def test_set(self):
    return self.x_test, self.y_test
  
num_labeled = 100
dataset = Dataset(num_labeled)




In [ ]:
#生成器

def build_generator(z_dim):

  model = Sequential()
  model.add(Dense(256*7*7, input_dim=z_dim))
  model.add(Reshape((7, 7, 256)))

  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

  model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

  model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding="same"))
  model.add(Activation("tanh"))

  return model

In [ ]:
#識別器

def build_discriminator_net(img_shape):

  model = Sequential()
  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(LeakyReLU(alpha=0.01))

  model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

  model.add(Conv2D(128, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(num_classes))

  return model

def build_discriminator_supervised(discriminator_net):
  
  model = Sequential()
  model.add(discriminator_net)
  model.add(Activation("softmax"))

  return model

def build_discriminator_unsupervised(discriminator_net):
  
  model = Sequential()
  model.add(discriminator_net)
  def predict(x):
    '''
    10ニューロンの出力大きい（つまりニューロンが入力画像を本物だと知覚した)時、predictionは1.0に近づく。
    逆に、10ニューロンの出力が全て小さい（つまりニューロンが入力画像を偽物だと知覚した）時、predictionは0に近づく。
    '''
    prediction = 1.0 - (1.0 /(K.sum(K.exp(x), axis=-1, keepdims=True) + 1.0))
    return prediction
  model.add(Lambda(predict))

  return model  

In [ ]:
#モデルの構築
def build_gan(generator, discriminator):

    model = Sequential()

    model.add(generator)
    model.add(discriminator)

    return model

discriminator_net = build_discriminator_net(img_shape)

# 教師あり学習のコンパイル
discriminator_supervised = build_discriminator_supervised(discriminator_net)
discriminator_supervised.compile(loss='categorical_crossentropy',
                                 metrics=['accuracy'],
                                 optimizer=Adam())

# 教師なし学習のコンパイル
discriminator_unsupervised = build_discriminator_unsupervised(discriminator_net)
discriminator_unsupervised.compile(loss='binary_crossentropy',
                                   optimizer=Adam())

#生成器のコンパイル
generator = build_generator(z_dim)
discriminator_unsupervised.trainable = False

gan = build_gan(generator, discriminator_unsupervised)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
#訓練アルゴリズム

supervised_losses = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):

  real = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for iteration in range(iterations):

    #ラベル付きサンプルの取得
    imgs, labels = dataset.batch_labeled(batch_size)
    labels = to_categorical(labels, num_classes=num_classes)

    #ラベルなしサンプルの取得
    imgs_unlabeled = dataset.batch_unlabeled(batch_size)

    #偽画像バッチの作成
    z = np.random.normal(0, 1, ((batch_size, z_dim)))
    gen_imgs = generator.predict(z)

    d_loss_supervised, accuracy = discriminator_supervised.train_on_batch(imgs, labels)
    d_loss_real = discriminator_unsupervised.train_on_batch(imgs_unlabeled, real)

    d_loss_fake = discriminator_unsupervised.train_on_batch(gen_imgs, fake)

    d_loss_unsupervised = 0.5 * np.add(d_loss_real, d_loss_fake)

    z = np.random.normal(0, 1, (batch_size, z_dim))
    gen_imgs = generator.predict(z)

    g_loss = gan.train_on_batch(z, real)

    if (iteration + 1) % sample_interval == 0:
      supervised_losses.append(d_loss_supervised)
      iteration_checkpoints.append(iteration+1)
      print(
                "%d [D loss supervised: %.4f, acc.: %.2f%%] [D loss unsupervised: %.4f] [G loss: %f]"
                % (iteration + 1, d_loss_supervised, 100 * accuracy,
                   d_loss_unsupervised, g_loss))




In [ ]:
iterations = 8000
batch_size = 32
sample_interval = 800

train(iterations, batch_size, sample_interval)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


800 [D loss supervised: 0.0045, acc.: 100.00%] [D loss unsupervised: 0.0361] [G loss: 3.187121]
1600 [D loss supervised: 0.0027, acc.: 100.00%] [D loss unsupervised: 0.2585] [G loss: 5.545338]
2400 [D loss supervised: 0.0004, acc.: 100.00%] [D loss unsupervised: 0.3236] [G loss: 3.961680]
3200 [D loss supervised: 0.0015, acc.: 100.00%] [D loss unsupervised: 0.2342] [G loss: 5.902021]
4000 [D loss supervised: 0.0004, acc.: 100.00%] [D loss unsupervised: 0.1315] [G loss: 4.544368]
4800 [D loss supervised: 0.0006, acc.: 100.00%] [D loss unsupervised: 0.4782] [G loss: 5.093963]
5600 [D loss supervised: 0.0006, acc.: 100.00%] [D loss unsupervised: 0.3707] [G loss: 4.130798]
6400 [D loss supervised: 0.0002, acc.: 100.00%] [D loss unsupervised: 0.6565] [G loss: 3.110205]
7200 [D loss supervised: 0.0009, acc.: 100.00%] [D loss unsupervised: 0.3305] [G loss: 3.829001]
8000 [D loss supervised: 0.0001, acc.: 100.00%] [D loss unsupervised: 0.2278] [G loss: 4.093932]


In [ ]:
x, y = dataset.test_set()
y = to_categorical(y, num_classes=num_classes)

# Compute classification accuracy on the test set
_, accuracy = discriminator_supervised.evaluate(x, y)
print("Test Accuracy: %.2f%%" % (100 * accuracy))

10000/10000 [==============================] - 1s 138us/step
Test Accuracy: 90.54%


In [ ]:
x, y = dataset.training_set()
y = to_categorical(y, num_classes=num_classes)

# Compute classification accuracy on the training set
_, accuracy = discriminator_supervised.evaluate(x, y)
print("Training Accuracy: %.2f%%" % (100 * accuracy))

100/100 [==============================] - 0s 483us/step
Training Accuracy: 100.00%
